In [1]:
import os
import csv
import gzip
import glob
import pathlib
import argparse
import datetime

In [2]:
import logging
########## logging
# create logger with 'spam_application'
logger = logging.getLogger('notebook')
logger.setLevel(logging.DEBUG)

# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('[%(asctime)s][%(levelname)s]: %(message)s')
ch.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(ch)
##########

In [3]:
import progressbar
progressbar.streams.wrap_stderr()

[2018-05-20 08:07:07,718][DEBUG]: input_files_count: 10
[2018-05-20 08:07:07,749][DEBUG]: input_file: ../data/test/pagecounts-20071210-070000.gz
[2018-05-20 08:07:07,753][INFO]: Processing file: ../data/test/pagecounts-20071210-070000.gz
[2018-05-20 08:07:10,373][INFO]: Added DataFrame for file ../data/test/pagecounts-20071210-070000.gz to list
[2018-05-20 08:07:10,374][DEBUG]: input_file: ../data/test/pagecounts-20071210-000000.gz
[2018-05-20 08:07:10,375][INFO]: Processing file: ../data/test/pagecounts-20071210-000000.gz
[2018-05-20 08:07:10,438][INFO]: Added DataFrame for file ../data/test/pagecounts-20071210-000000.gz to list
[2018-05-20 08:07:10,441][DEBUG]: input_file: ../data/test/pagecounts-20071210-080000.gz
[2018-05-20 08:07:10,442][INFO]: Processing file: ../data/test/pagecounts-20071210-080000.gz
[2018-05-20 08:07:10,504][INFO]: Added DataFrame for file ../data/test/pagecounts-20071210-080000.gz to list
[2018-05-20 08:07:10,505][DEBUG]: input_file: ../data/test/pagecounts-2

In [4]:
import findspark
findspark.init()

In [5]:
import pyspark
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, TimestampType
from pyspark.sql import functions
from pyspark.sql.functions import lit

In [6]:
sc = pyspark.SparkContext(appName="merge-pagecounts")
sqlctx = pyspark.SQLContext(sc)

In [7]:
schema = StructType([StructField("lang", StringType(), False),
                     StructField("page", StringType(), False),
                     StructField("views", IntegerType(), False),
                     StructField("reqbytes", IntegerType(), False)])

In [8]:
def unionAll(*dfs):
    first, *_ = dfs  # Python 3.x, for 2.x you'll have to unpack manually
    return first.sql_ctx.createDataFrame(
        first.sql_ctx._sc.union([df.rdd for df in dfs]),
        first.schema
    )


def date_parser(timestamp):
    return datetime.datetime.strptime(timestamp, '%Y%m%d-%H%M%S')

In [9]:
pathfile = "../data/test/pagecounts-20071210-0*.gz"

In [10]:
input_files_count = len([f for f in glob.iglob(pathfile)])
input_files = glob.iglob(pathfile)

# input_files = ["data/input/sorted_time/2007-12/pagecounts-20071210-000000.gz",
#                "data/input/sorted_time/2007-12/pagecounts-20071210-010000.gz"
#                ]
# input_files_count = len(input_files)

logger.debug('input_files_count: {}'.format(input_files_count))

In [11]:
if input_files_count < 1:
    logger.warn('No input files match: exiting')
    exit(0)

In [12]:
input_files_count

10

In [13]:
list_dfs = list()
with progressbar.ProgressBar(max_value=input_files_count) as bar:
    for input_file in input_files:
        logger.debug('input_file: {}'.format(input_file))

        timestamp = date_parser(os.path.basename(input_file)
                                       .replace('pagecounts-','')
                                       .replace('.gz',''))

        logger.info('Processing file: {}'.format(input_file))
        tmp_spark_df = sqlctx.read.csv(
                            input_file,
                            header=False,
                            schema=schema,
                            sep=' ')

        tmp_spark_df = tmp_spark_df.withColumn("timestamp", lit(timestamp))
        list_dfs.append(tmp_spark_df)
        del tmp_spark_df

        logger.info('Added DataFrame for file {} to list'.format(input_file))

In [14]:
assert len(list_dfs) >= 1, 'There should be at least one DataFrame'

In [15]:
if len(list_dfs) > 1:
    logger.info('Union of all Spark DataFrames.')
    df = unionAll(*list_dfs)
    logger.info('Spark DataFrame created')
else:
    df = list_dfs[0]

In [16]:
df.count()

122

In [17]:
logger.info('Dropping column "reqbytes" from DataFrame')
df = df.drop('reqbytes')
logger.info('Dropped column "reqbytes" from DataFrame')

In [18]:
df.dtypes

[('lang', 'string'),
 ('page', 'string'),
 ('views', 'int'),
 ('timestamp', 'timestamp')]

In [19]:
grouped_daily_df = (df.select(['lang',
                               'page',
                               functions.date_format('timestamp','yyyy-MM-dd')\
                                        .alias('day'),
                               'views'])
                      .groupby(['lang','page','day'])
                      .sum('views')
                      )

In [20]:
grouped_daily_df.show(10)

+----+--------------------+----------+----------+
|lang|                page|       day|sum(views)|
+----+--------------------+----------+----------+
|  en|Albert_Camus#Furt...|2007-12-10|         1|
|  en|Albert_Camuscolum...|2007-12-10|         1|
|  en|    Albert_Carnesale|2007-12-10|         9|
|  en|        Albert_Camus|2007-12-10|      1362|
|  en|      Albert_Bunjaku|2007-12-10|         1|
|  en|      Albert_Cadwell|2007-12-10|         1|
|  en|       Albert_Caquot|2007-12-10|         4|
|  en|     Albert_C._Field|2007-12-10|         1|
|  en|Albert_Camus#Oppo...|2007-12-10|         1|
|  en|Albert_CamusNon-f...|2007-12-10|         1|
+----+--------------------+----------+----------+
only showing top 10 rows



In [21]:
new_schema = StructType([StructField("lang", StringType(), False),
                         StructField("page", StringType(), False),
                         StructField("day", StringType(), False),
                         StructField("enc", StringType(), False)])

In [22]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd

hour_to_letter = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O',
                  'P','Q','R','S','T','U','V','W','X']

@pandas_udf(new_schema, PandasUDFType.GROUPED_MAP)
def concat_hours(x):
    view_hours = x['hour'].tolist()
    view_views = x['views'].tolist()

    view_hours_letters = [hour_to_letter[h] for h in view_hours]

    encoded_views = [l + str(h)
                     for l, h in sorted(zip(view_hours_letters,view_views))]
    encoded_views_string = ''.join(encoded_views)

    # return pd.DataFrame({'page': x.page, 'lang': x.lang,'day': x.day, 'enc': encoded_views_string}, index=[x.index[0]])
    return pd.DataFrame({'enc': x.page, 'day': x.lang, 'lang': x.day, 'page': encoded_views_string}, index=[x.index[0]])

In [25]:
from pyspark.sql import functions
grouped_hours_df = (df.select(['lang',
                               'page',
                               functions.date_format('timestamp','yyyy-MM-dd').alias('day'), 
                               functions.hour('timestamp').alias('hour'), 
                               'views'
                               ])
                      .groupby(['lang','page','day'])
                      )

In [26]:
grouped_hours_df = (grouped_hours_df.apply(concat_hours)
                                    .dropDuplicates()
                                    )

In [27]:
grouped_hours_df.show()

+----+--------------------+----------+--------------------+
|lang|                page|       day|                 enc|
+----+--------------------+----------+--------------------+
|  en|        Albert_Cahen|2007-12-10|                  J1|
|  en|     Albert_Campbell|2007-12-10|            A1D1F1I1|
|  en|Albert_C._L._G._G...|2007-12-10|                  J1|
|  en|Albert_Cardinal_V...|2007-12-10|                  E1|
|  en|        Albert_Canal|2007-12-10|            A2C1G2I1|
|  en|Albert_Camus#Oppo...|2007-12-10|                  E1|
|  en|       Albert_Brooks|2007-12-10|                 G16|
|  en|      Albert_Cashier|2007-12-10|          A2B1D7E1H2|
|  en|     Albert_Castillo|2007-12-10|              B1E1H1|
|  en|    Albert_C._Outler|2007-12-10|                  H2|
|  en|   Albert_Carotenuto|2007-12-10|                D1H1|
|  en|     Albert_C._Field|2007-12-10|                  G1|
|  en|Albert_Camus#Summ...|2007-12-10|                  D1|
|  en|       Albert_Cavens|2007-12-10|  

In [29]:
grouped_daily_df.show(10)

+----+--------------------+----------+----------+
|lang|                page|       day|sum(views)|
+----+--------------------+----------+----------+
|  en|Albert_Camus#Furt...|2007-12-10|         1|
|  en|Albert_Camuscolum...|2007-12-10|         1|
|  en|    Albert_Carnesale|2007-12-10|         9|
|  en|        Albert_Camus|2007-12-10|      1362|
|  en|      Albert_Bunjaku|2007-12-10|         1|
|  en|      Albert_Cadwell|2007-12-10|         1|
|  en|       Albert_Caquot|2007-12-10|         4|
|  en|     Albert_C._Field|2007-12-10|         1|
|  en|Albert_Camus#Oppo...|2007-12-10|         1|
|  en|Albert_CamusNon-f...|2007-12-10|         1|
+----+--------------------+----------+----------+
only showing top 10 rows



In [32]:
from pyspark.sql.functions import col
daily = grouped_daily_df.select([col('lang').alias('daily_lang'),
                                 col('page').alias('daily_page'),
                                 col('day').alias('daily_day'),
                                 col('sum(views)').alias('daily_sum_views'),                                 
                                 ])

In [33]:
daily.show()

+----------+--------------------+----------+---------------+
|daily_lang|          daily_page| daily_day|daily_sum_views|
+----------+--------------------+----------+---------------+
|        en|Albert_Camus#Furt...|2007-12-10|              1|
|        en|Albert_Camuscolum...|2007-12-10|              1|
|        en|    Albert_Carnesale|2007-12-10|              9|
|        en|        Albert_Camus|2007-12-10|           1362|
|        en|      Albert_Bunjaku|2007-12-10|              1|
|        en|      Albert_Cadwell|2007-12-10|              1|
|        en|       Albert_Caquot|2007-12-10|              4|
|        en|     Albert_C._Field|2007-12-10|              1|
|        en|Albert_Camus#Oppo...|2007-12-10|              1|
|        en|Albert_CamusNon-f...|2007-12-10|              1|
|        en|      Albert_Celades|2007-12-10|              6|
|        en|      Albert_Cashier|2007-12-10|             13|
|        en|      Albert_Calland|2007-12-10|              5|
|        en|       Alber

In [35]:
cond = [daily.daily_lang == grouped_hours_df.lang,
        daily.daily_page == grouped_hours_df.page,
        daily.daily_day == grouped_hours_df.day]
final = daily.join(grouped_hours_df, cond).select(['daily_lang','daily_page','daily_day', 'daily_sum_views', 'enc'])

In [36]:
final.dropDuplicates().show()

+----------+--------------------+----------+---------------+--------------------+
|daily_lang|          daily_page| daily_day|daily_sum_views|                 enc|
+----------+--------------------+----------+---------------+--------------------+
|        en|Albert_Camus#Furt...|2007-12-10|              1|                  E1|
|        en|Albert_Camuscolum...|2007-12-10|              1|                  J1|
|        en|    Albert_Carnesale|2007-12-10|              9|                G4H5|
|        en|      Albert_Bunjaku|2007-12-10|              1|                  A1|
|        en|        Albert_Camus|2007-12-10|           1362|A150B148C197D173E...|
|        en|      Albert_Cadwell|2007-12-10|              1|                  D1|
|        en|     Albert_C._Field|2007-12-10|              1|                  G1|
|        en|Albert_Camus#Oppo...|2007-12-10|              1|                  E1|
|        en|       Albert_Caquot|2007-12-10|              4|            C1E1G1I1|
|        en|Albe